In [2]:
from keras.applications import inception_v3
from keras import backend as K


In [3]:
K.set_learning_phase(0) #Set learning phase to test. Prevent model learning for safety reasons.

In [4]:
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

87916544/87910968 [==============================] - 41s 0us/step


In [6]:
# Layers could be from mixed0 up to mixed10
# Check names with command model.summary()
layer_contribution = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 1.5
}

In [19]:
K.cast(layer_dict['mixed5'].output, 'float32')

<tf.Tensor 'mixed5/concat:0' shape=(?, ?, ?, 768) dtype=float32>

In [20]:
# Loss calculation (layers outputs + L2_norm)
layer_dict = dict([(layer.name, layer) for layer in model.layers])
loss = K.variable(0.)

for layer_name in layer_contribution:
    coeff = layer_contribution[layer_name]
    # Get layer output
    activation = layer_dict[layer_name].output
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss += coeff * K.sum(K.square(activation)) / scaling

<tf.Tensor 'add_3:0' shape=() dtype=float32>

In [8]:
layer_dict

{'input_1': <keras.engine.input_layer.InputLayer at 0x7f64af6b6c18>,
 'conv2d_1': <keras.layers.convolutional.Conv2D at 0x7f64aed8e470>,
 'batch_normalization_1': <keras.layers.normalization.BatchNormalization at 0x7f64af84ba58>,
 'activation_1': <keras.layers.core.Activation at 0x7f64af6c9da0>,
 'conv2d_2': <keras.layers.convolutional.Conv2D at 0x7f64af578128>,
 'batch_normalization_2': <keras.layers.normalization.BatchNormalization at 0x7f64af3374e0>,
 'activation_2': <keras.layers.core.Activation at 0x7f64af337d30>,
 'conv2d_3': <keras.layers.convolutional.Conv2D at 0x7f64af1ecf60>,
 'batch_normalization_3': <keras.layers.normalization.BatchNormalization at 0x7f64af03add8>,
 'activation_3': <keras.layers.core.Activation at 0x7f64af03af60>,
 'max_pooling2d_1': <keras.layers.pooling.MaxPooling2D at 0x7f64c664d2b0>,
 'conv2d_4': <keras.layers.convolutional.Conv2D at 0x7f64af5b1a20>,
 'batch_normalization_4': <keras.layers.normalization.BatchNormalization at 0x7f64af74cb38>,
 'activatio

In [5]:
 #mixed0 - mixed10

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
_______________________________________________________________________________________